Parallel Computing with Python
================================

**[Rodrigo Nemmen](http://rodrigonemmen.com), IAG USP**

This IPython notebook illustrates a few simple ways of doing parallel computing.

Practical examples included:

1. Parallel function mapping to a list of arguments (multiprocessing module)
2. Parallel execution of array function (scatter/gather) + parallel execution of scripts
4. Easy parallel Monte Carlo (parallel magics)

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 1. Mapping a model to a grid of parameters 

<!---
Inspired on "useful parallel".
-->

Uses the *multiprocessing* module that comes by default with python, i.e. method independent of IPython.

Idea: you have a function $f(\mathbf{x},\mathbf{y})$ of two parameters (e.g., $f$ may represent your model) stored in the arrays $(\mathbf{x},\mathbf{y})$. Given the arrays $\mathbf{x}$ and $\mathbf{y}$, you want to compute the values of $f(\mathbf{x},\mathbf{y})$. Let's assume for simplicity that there is no dependence on the neighbours. This is an embarassingly parallel problem.

<!---
### TODO

* Random sampling of parameter space if desired
-->

In [2]:
import multiprocessing
import scipy
import sample

Time wasting function that depends on two parameters. Here, I generate 1E5 random numbers based on the normal distribution and then sum them. The two parameters are $\mu$ and $\sigma$.

In [3]:
#def f(z):
#    x=z[1]*scipy.random.standard_normal(100000)+z[0]
#    return x.sum()

Arrays of input parameters. You could easily modify this to take as input a matrix, not two arrays.

In [4]:
n=3000
X=numpy.linspace(-1,1,n) # mean
Y=numpy.linspace(0.1,1,n) # std. dev.

In [4]:
# creates list of arguments [Xi, Yi]
pargs=[]# this is a list of lists!
for i in range(X.size):
    pargs.append([X[i],Y[i]])

Parallel execution. Check out all the cores being used with a tool like [htop](http://hisham.hm/htop/).

In [5]:
ncores=multiprocessing.cpu_count() # number of cores
print(ncores)
pool = multiprocessing.Pool(processes=ncores) # initializes parallel engine

4


In [13]:
%%time
t=pool.map(sample.f, pargs)# parallel function map

NameError: name 'scipy' is not defined

In [ ]:
pool.close()# close the parallel engine

Serial execution

In [ ]:
%time t=map(f[pargs])

If you want to convert the list to an array use `y=array(t)`. Also note that there is a similar map method for `ipyparallel`.

# 2. Parallel execution of array function

Uses [`ipyparallel`](https://ipyparallel.readthedocs.io/en/latest/index.html). Consider a function $f(x)$ which takes an array $x$ containing the grid of input parameters. We want to split the function calls ("split the array") to the different cores in our machine:

![test](http://computing.llnl.gov/tutorials/parallel_comp/images/array_proc1.gif)
![test](http://computing.llnl.gov/tutorials/parallel_comp/images/array_proc2.gif)

Make sure you start the parallel engines

![Click the clusters tab](https://raw.githubusercontent.com/rsnemmen/parallel-python-tutorial/master/notebook01.png)
![Start the parallel engine](https://raw.githubusercontent.com/rsnemmen/parallel-python-tutorial/master/notebook02.png)

Alternatively, you can start the engines from the command-line:

    $ ipcluster start -n 4

Our time-waster function $f(x)$ that can be applied to an array of integers

In [7]:
# test if n is prime
def isprime(n):
    for i in range(3, n):
        if n % i == 0:
            return False
    return True

# tests each element of an array if it is prime
def f(x):
    return map(isprime,x) 

Generates big array (10k elements) of random integers between 0 and 100000

In [8]:
x = scipy.random.randint(0,100000, (10000,)) 

Serial execution

In [11]:
%time y=f(x)

Wall time: 0 ns


Now explain how IPython parallel works **(here I show a slide)**. See documentation at the end of the notebook for details.

In [12]:
import ipyparallel
client = ipyparallel.Client()

We are going to use the direct view, which means that commands always run on all nodes. This as opposed to a balanced view, which asynchronously executes code on nodes which are idle. In addition, we are going to turn blocking on. This means that jobs will block further execution until all nodes have finished.

In [13]:
direct = client[:]
direct.block = True

Splits the input array $x$ between the cores

In [14]:
direct.scatter('x',x)

Verify that the array was indeed divided equally

In [15]:
direct['x.size']

[2500, 2500, 2500, 2500]

In [16]:
direct['x']

[array([59457, 38694, 47592, ..., 79304, 11167, 87940]),
 array([61139, 58524, 30704, ..., 62571,  3745, 57039]),
 array([51329, 52135, 32463, ..., 95304,  9332, 31944]),
 array([33767, 14543, 84441, ..., 67555, 84692, 34277])]

Let's try to apply the function in each different core

In [17]:
%%px
y=f(x)

CompositeError: one or more exceptions from call to method: execute
[0:execute]: NameError: name 'f' is not defined
[1:execute]: NameError: name 'f' is not defined
[2:execute]: NameError: name 'f' is not defined
[3:execute]: NameError: name 'f' is not defined

Why the errors above? Because each core does not see the local engine. They work as separate machines and you have to load all variables and modules in each engine. That's easy.

In [18]:
%%file myscript.py
# test if n is prime
def isprime(n):
    for i in range(3, n):
        if n % i == 0:
            return False
    return True

# tests each element of an array if it is prime
def f(x):
    return map(isprime,x) 

Writing myscript.py


Execute code which defines the methods on the different engines

In [19]:
direct.run("myscript.py")

<AsyncResult: execute:finished>

Now compute the "model grid" correctly

In [20]:
%%time
%px y=f(x)

Wall time: 22.1 ms


Alternatively to the command above, you could use

    direct.apply(f,x)
or

    direct.execute('y=f(x)')

Now we have the separate arrays $y$ containing the results on each engine. How to get it back to the local engine?

In [21]:
%%px 
import numpy
numpy.size(y)

Out[0:3]: 1

Out[1:3]: 1

Out[2:3]: 1

Out[3:3]: 1

In [22]:
y=direct.gather('y')

We have the array magically reassembled back in the local engine.  :)

# 3. Easy parallel Monte Carlo

Suppose you need to do 100k Monte Carlo simulations. Wouldn't it be great if you could easily split them among your (hopefully many) cores?

In this example, I will perform 100k realizations of a 300x300 array of random floats.

In [23]:
# number of desired random sets
nboot=100000

# number of sets that will be computed by each engine
n=nboot/size(client.ids)

Passes variables to the engines

In [24]:
direct.push(dict(n=n))

with direct.sync_imports():
    import scipy

importing scipy on engine(s)


Now everything below is executed in parallel! (IPython magic)

<!---
Have a look also at the %autopx command.
-->

In [27]:
%%time
%%px

for i in range(int(n)):
    x = scipy.random.random((300,300)) # 300x300 array of floats (values in the range [0,1)   )

Wall time: 50.5 s


For comparison, how long does it take to do the same simulation in serial mode?

In [28]:
%%time
for i in range(nboot):
    x = scipy.random.random((300,300)) # 100x100 array of floats (values in the range [0,1)   )

Wall time: 1min 31s


# Useful reference

## IPython video tutorials

* [IPython in depth: high productivity interactive and parallel python - PyCon 2014](http://youtu.be/XFw1JVXKJss)
* [The IPython Notebook Revolution](http://youtu.be/t_TzRaK9kpU)

## Parallel computing

### General

* [Introduction to parallel programming](https://computing.llnl.gov/tutorials/parallel_comp)

### Python

* [AstroPython blog](http://astropython.blogspot.com.br), maintained (not so often anymore) by the [speaker](http://rodrigonemmen.com)
* [Using IPython for parallel computing](https://ipyparallel.readthedocs.io/en/latest/index.html). Documentation of IPython.parallel
* [Parallel computing with IPython](http://www.astro.washington.edu/users/vanderplas/Astr599/notebooks/21_IPythonParallel). Nice tutorial
* [Simple python parallelism](http://scottsievert.github.io/blog/2014/07/30/simple-python-parallelism/) and [easily distributing a parallel IPy Notebook on a cluster](http://twiecki.github.io/blog/2014/02/24/ipython-nb-cluster/). Some ideas on making a function parallel easily

### Executing general (non-python) jobs in parallel

* [Submit non-python commands to a running IPython cluster](https://gist.github.com/zonca/8994544)
* [GNU Parallel](http://www.gnu.org/software/parallel/)
* [xjobs](http://www.maier-komor.de/xjobs.html)